In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    ###################################################### ~Q(s,~a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # Q-learning
    #aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs))
    aloss2 += -tf.reduce_mean(aQs_logits) # max
    aloss2 += -tf.reduce_mean(aQs2_logits) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e2)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0 
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training
            while True:
                batch = memory.sample(batch_size=batch_size)
                rates = np.array([each[5] for each in batch])
                states = np.array([each[0] for each in batch])[rates > 0]
                actions = np.array([each[1] for each in batch])[rates > 0]
                next_states = np.array([each[2] for each in batch])[rates > 0]
                rewards = np.array([each[3] for each in batch])[rates > 0]
                dones = np.array([each[4] for each in batch])[rates > 0]
                rates = np.array([each[5] for each in batch])[rates > 0]
                if len(dones) == batch_size:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            #print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:0.7034 eloss:3.8038 aloss2:1.9327 exploreP:0.9985
Episode:1 meanR:18.0000 R:21.0000 rate:0.0420 aloss:0.7134 eloss:3.7880 aloss2:1.9536 exploreP:0.9964
Episode:2 meanR:17.0000 R:15.0000 rate:0.0300 aloss:0.7011 eloss:3.7725 aloss2:1.9586 exploreP:0.9950
Episode:3 meanR:16.2500 R:14.0000 rate:0.0280 aloss:0.7064 eloss:3.7593 aloss2:1.9546 exploreP:0.9936
Episode:4 meanR:18.2000 R:26.0000 rate:0.0520 aloss:0.7094 eloss:3.7415 aloss2:1.9808 exploreP:0.9910
Episode:5 meanR:19.3333 R:25.0000 rate:0.0500 aloss:0.7067 eloss:3.7208 aloss2:1.9980 exploreP:0.9886
Episode:6 meanR:20.7143 R:29.0000 rate:0.0580 aloss:0.7032 eloss:3.7109 aloss2:2.0020 exploreP:0.9857
Episode:7 meanR:19.8750 R:14.0000 rate:0.0280 aloss:0.7073 eloss:3.7085 aloss2:2.0140 exploreP:0.9844
Episode:8 meanR:20.3333 R:24.0000 rate:0.0480 aloss:0.7058 eloss:3.6806 aloss2:2.0301 exploreP:0.9820
Episode:9 meanR:21.8000 R:35.0000 rate:0.0700 aloss:0.7062 eloss:3.6680 aloss2:2.0

Episode:80 meanR:20.3086 R:18.0000 rate:0.0360 aloss:0.7139 eloss:2.5910 aloss2:3.6880 exploreP:0.8498
Episode:81 meanR:20.1829 R:10.0000 rate:0.0200 aloss:0.7214 eloss:2.5780 aloss2:3.7182 exploreP:0.8490
Episode:82 meanR:20.1205 R:15.0000 rate:0.0300 aloss:0.7228 eloss:2.5673 aloss2:3.7469 exploreP:0.8477
Episode:83 meanR:20.0357 R:13.0000 rate:0.0260 aloss:0.7192 eloss:2.5542 aloss2:3.7876 exploreP:0.8466
Episode:84 meanR:19.9529 R:13.0000 rate:0.0260 aloss:0.7133 eloss:2.5384 aloss2:3.8091 exploreP:0.8456
Episode:85 meanR:19.8953 R:15.0000 rate:0.0300 aloss:0.7084 eloss:2.5252 aloss2:3.8461 exploreP:0.8443
Episode:86 meanR:20.0460 R:33.0000 rate:0.0660 aloss:0.7151 eloss:2.5148 aloss2:3.8889 exploreP:0.8416
Episode:87 meanR:20.0568 R:21.0000 rate:0.0420 aloss:0.7157 eloss:2.4873 aloss2:3.9522 exploreP:0.8398
Episode:88 meanR:20.0112 R:16.0000 rate:0.0320 aloss:0.7214 eloss:2.4860 aloss2:3.9711 exploreP:0.8385
Episode:89 meanR:19.9444 R:14.0000 rate:0.0280 aloss:0.7164 eloss:2.4625 

Episode:159 meanR:20.7900 R:12.0000 rate:0.0240 aloss:0.6934 eloss:4.2344 aloss2:3.2963 exploreP:0.7177
Episode:160 meanR:20.7200 R:12.0000 rate:0.0240 aloss:0.6924 eloss:4.2127 aloss2:3.3352 exploreP:0.7168
Episode:161 meanR:20.7200 R:10.0000 rate:0.0200 aloss:0.6952 eloss:4.2992 aloss2:3.3030 exploreP:0.7161
Episode:162 meanR:20.6500 R:13.0000 rate:0.0260 aloss:0.6965 eloss:4.0922 aloss2:3.4186 exploreP:0.7152
Episode:163 meanR:21.0600 R:53.0000 rate:0.1060 aloss:0.6950 eloss:4.1382 aloss2:3.4052 exploreP:0.7115
Episode:164 meanR:20.9800 R:11.0000 rate:0.0220 aloss:0.6942 eloss:4.0991 aloss2:3.4585 exploreP:0.7107
Episode:165 meanR:21.0200 R:15.0000 rate:0.0300 aloss:0.6945 eloss:4.0863 aloss2:3.4754 exploreP:0.7097
Episode:166 meanR:20.9600 R:19.0000 rate:0.0380 aloss:0.6934 eloss:3.9819 aloss2:3.5615 exploreP:0.7083
Episode:167 meanR:21.0900 R:25.0000 rate:0.0500 aloss:0.6947 eloss:3.8750 aloss2:3.6410 exploreP:0.7066
Episode:168 meanR:21.0900 R:31.0000 rate:0.0620 aloss:0.6944 elo

Episode:239 meanR:19.9500 R:14.0000 rate:0.0280 aloss:0.7004 eloss:2.2097 aloss2:7.1673 exploreP:0.6144
Episode:240 meanR:19.8800 R:12.0000 rate:0.0240 aloss:0.7034 eloss:2.2647 aloss2:7.0623 exploreP:0.6137
Episode:241 meanR:19.8100 R:13.0000 rate:0.0260 aloss:0.6911 eloss:2.2219 aloss2:7.1227 exploreP:0.6129
Episode:242 meanR:19.8300 R:18.0000 rate:0.0360 aloss:0.7025 eloss:2.2944 aloss2:7.1281 exploreP:0.6118
Episode:243 meanR:19.9800 R:25.0000 rate:0.0500 aloss:0.6981 eloss:2.3550 aloss2:7.0232 exploreP:0.6103
Episode:244 meanR:19.7300 R:12.0000 rate:0.0240 aloss:0.6991 eloss:2.3285 aloss2:7.0169 exploreP:0.6096
Episode:245 meanR:19.9000 R:27.0000 rate:0.0540 aloss:0.6968 eloss:2.3733 aloss2:6.9961 exploreP:0.6079
Episode:246 meanR:20.0000 R:23.0000 rate:0.0460 aloss:0.6968 eloss:2.4493 aloss2:6.9045 exploreP:0.6066
Episode:247 meanR:20.0400 R:28.0000 rate:0.0560 aloss:0.7001 eloss:2.5191 aloss2:6.7306 exploreP:0.6049
Episode:248 meanR:20.0400 R:10.0000 rate:0.0200 aloss:0.7072 elo

Episode:319 meanR:17.4500 R:14.0000 rate:0.0280 aloss:0.6982 eloss:2.0495 aloss2:8.0039 exploreP:0.5364
Episode:320 meanR:17.4500 R:12.0000 rate:0.0240 aloss:0.7042 eloss:2.0894 aloss2:8.0005 exploreP:0.5357
Episode:321 meanR:17.2800 R:10.0000 rate:0.0200 aloss:0.7034 eloss:2.0639 aloss2:7.9945 exploreP:0.5352
Episode:322 meanR:17.1500 R:9.0000 rate:0.0180 aloss:0.7079 eloss:2.0432 aloss2:8.0264 exploreP:0.5347
Episode:323 meanR:17.2200 R:18.0000 rate:0.0360 aloss:0.7189 eloss:2.1384 aloss2:7.9862 exploreP:0.5338
Episode:324 meanR:17.0700 R:8.0000 rate:0.0160 aloss:0.7036 eloss:2.1274 aloss2:7.9856 exploreP:0.5334
Episode:325 meanR:17.1800 R:22.0000 rate:0.0440 aloss:0.7085 eloss:2.0677 aloss2:8.0142 exploreP:0.5322
Episode:326 meanR:17.3300 R:23.0000 rate:0.0460 aloss:0.7087 eloss:2.0825 aloss2:8.0318 exploreP:0.5310
Episode:327 meanR:17.1700 R:15.0000 rate:0.0300 aloss:0.7105 eloss:2.0986 aloss2:8.0414 exploreP:0.5302
Episode:328 meanR:16.8100 R:10.0000 rate:0.0200 aloss:0.7195 eloss

...

Episode:8099 meanR:11.1500 R:10.0000 rate:0.0200 aloss:0.4711 eloss:2.6404 aloss2:8.1894 exploreP:0.0102
Episode:8100 meanR:11.1500 R:10.0000 rate:0.0200 aloss:0.4539 eloss:2.6060 aloss2:8.2000 exploreP:0.0102
Episode:8101 meanR:11.4000 R:35.0000 rate:0.0700 aloss:0.4570 eloss:2.5456 aloss2:8.2607 exploreP:0.0102
Episode:8102 meanR:11.0900 R:9.0000 rate:0.0180 aloss:0.4569 eloss:2.6234 aloss2:8.1489 exploreP:0.0102
Episode:8103 meanR:11.0800 R:9.0000 rate:0.0180 aloss:0.4791 eloss:2.5600 aloss2:8.2219 exploreP:0.0102
Episode:8104 meanR:11.2100 R:22.0000 rate:0.0440 aloss:0.4583 eloss:2.5428 aloss2:8.2392 exploreP:0.0102
Episode:8105 meanR:11.1900 R:10.0000 rate:0.0200 aloss:0.4598 eloss:2.5769 aloss2:8.1583 exploreP:0.0102
Episode:8106 meanR:11.4500 R:36.0000 rate:0.0720 aloss:0.4555 eloss:2.5459 aloss2:8.1531 exploreP:0.0102
Episode:8107 meanR:11.4400 R:9.0000 rate:0.0180 aloss:0.4440 eloss:2.5294 aloss2:8.0997 exploreP:0.0102
Episode:8108 meanR:11.5700 R:22.0000 rate:0.0440 aloss:0.4

Episode:8178 meanR:13.6000 R:22.0000 rate:0.0440 aloss:0.5155 eloss:2.6919 aloss2:7.9899 exploreP:0.0102
Episode:8179 meanR:13.4900 R:10.0000 rate:0.0200 aloss:0.5311 eloss:2.7461 aloss2:7.9732 exploreP:0.0102
Episode:8180 meanR:13.5900 R:20.0000 rate:0.0400 aloss:0.5064 eloss:2.6701 aloss2:8.0530 exploreP:0.0102
Episode:8181 meanR:13.5800 R:9.0000 rate:0.0180 aloss:0.5236 eloss:2.6856 aloss2:7.9880 exploreP:0.0102
Episode:8182 meanR:13.5900 R:10.0000 rate:0.0200 aloss:0.5122 eloss:2.6660 aloss2:8.0408 exploreP:0.0102
Episode:8183 meanR:13.6000 R:10.0000 rate:0.0200 aloss:0.4845 eloss:2.6335 aloss2:8.1038 exploreP:0.0102
Episode:8184 meanR:13.7200 R:21.0000 rate:0.0420 aloss:0.4873 eloss:2.6274 aloss2:8.1443 exploreP:0.0102
Episode:8185 meanR:13.7400 R:10.0000 rate:0.0200 aloss:0.4948 eloss:2.6573 aloss2:8.1014 exploreP:0.0102
Episode:8186 meanR:13.5900 R:9.0000 rate:0.0180 aloss:0.4780 eloss:2.6343 aloss2:8.0807 exploreP:0.0102
Episode:8187 meanR:13.5900 R:10.0000 rate:0.0200 aloss:0.

Episode:8258 meanR:11.5800 R:10.0000 rate:0.0200 aloss:0.4483 eloss:2.5768 aloss2:7.9559 exploreP:0.0102
Episode:8259 meanR:11.5700 R:9.0000 rate:0.0180 aloss:0.4568 eloss:2.4854 aloss2:8.0631 exploreP:0.0102
Episode:8260 meanR:11.5700 R:10.0000 rate:0.0200 aloss:0.4230 eloss:2.5389 aloss2:8.0099 exploreP:0.0102
Episode:8261 meanR:11.4000 R:9.0000 rate:0.0180 aloss:0.4343 eloss:2.5500 aloss2:7.9476 exploreP:0.0102
Episode:8262 meanR:11.2500 R:9.0000 rate:0.0180 aloss:0.4343 eloss:2.5205 aloss2:7.9964 exploreP:0.0102
Episode:8263 meanR:11.2500 R:10.0000 rate:0.0200 aloss:0.4360 eloss:2.5927 aloss2:7.9229 exploreP:0.0102
Episode:8264 meanR:11.1100 R:10.0000 rate:0.0200 aloss:0.4345 eloss:2.5798 aloss2:7.9669 exploreP:0.0102
Episode:8265 meanR:10.9400 R:9.0000 rate:0.0180 aloss:0.4422 eloss:2.4934 aloss2:8.0292 exploreP:0.0102
Episode:8266 meanR:10.9300 R:9.0000 rate:0.0180 aloss:0.4408 eloss:2.5257 aloss2:7.9867 exploreP:0.0102
Episode:8267 meanR:10.9400 R:10.0000 rate:0.0200 aloss:0.419

Episode:8338 meanR:9.8400 R:8.0000 rate:0.0160 aloss:0.4827 eloss:2.4845 aloss2:8.2360 exploreP:0.0101
Episode:8339 meanR:9.8500 R:10.0000 rate:0.0200 aloss:0.4584 eloss:2.5462 aloss2:8.2519 exploreP:0.0101
Episode:8340 meanR:9.8600 R:10.0000 rate:0.0200 aloss:0.4469 eloss:2.5679 aloss2:8.2334 exploreP:0.0101
Episode:8341 meanR:9.8400 R:8.0000 rate:0.0160 aloss:0.4775 eloss:2.5221 aloss2:8.2299 exploreP:0.0101
Episode:8342 meanR:9.8600 R:10.0000 rate:0.0200 aloss:0.4407 eloss:2.5768 aloss2:8.1855 exploreP:0.0101
Episode:8343 meanR:9.8600 R:9.0000 rate:0.0180 aloss:0.4377 eloss:2.5870 aloss2:8.1659 exploreP:0.0101
Episode:8344 meanR:9.8500 R:9.0000 rate:0.0180 aloss:0.4515 eloss:2.5022 aloss2:8.2977 exploreP:0.0101
Episode:8345 meanR:9.8400 R:9.0000 rate:0.0180 aloss:0.4443 eloss:2.5779 aloss2:8.1847 exploreP:0.0101
Episode:8346 meanR:9.8300 R:9.0000 rate:0.0180 aloss:0.4373 eloss:2.6144 aloss2:8.1922 exploreP:0.0101
Episode:8347 meanR:9.8200 R:9.0000 rate:0.0180 aloss:0.4607 eloss:2.63

Episode:8419 meanR:10.4100 R:9.0000 rate:0.0180 aloss:0.4837 eloss:2.6042 aloss2:8.0854 exploreP:0.0101
Episode:8420 meanR:10.4300 R:10.0000 rate:0.0200 aloss:0.4683 eloss:2.6222 aloss2:8.0745 exploreP:0.0101
Episode:8421 meanR:10.5500 R:22.0000 rate:0.0440 aloss:0.4694 eloss:2.6704 aloss2:8.0439 exploreP:0.0101
Episode:8422 meanR:10.8100 R:35.0000 rate:0.0700 aloss:0.4771 eloss:2.6010 aloss2:8.0926 exploreP:0.0101
Episode:8423 meanR:10.8000 R:8.0000 rate:0.0160 aloss:0.4715 eloss:2.6365 aloss2:8.0228 exploreP:0.0101
Episode:8424 meanR:10.8000 R:10.0000 rate:0.0200 aloss:0.4858 eloss:2.6199 aloss2:8.0753 exploreP:0.0101
Episode:8425 meanR:10.8000 R:10.0000 rate:0.0200 aloss:0.4555 eloss:2.5375 aloss2:8.1847 exploreP:0.0101
Episode:8426 meanR:10.7800 R:8.0000 rate:0.0160 aloss:0.4716 eloss:2.6280 aloss2:8.0927 exploreP:0.0101
Episode:8427 meanR:10.7600 R:8.0000 rate:0.0160 aloss:0.4848 eloss:2.5227 aloss2:8.2548 exploreP:0.0101
Episode:8428 meanR:10.7600 R:9.0000 rate:0.0180 aloss:0.457

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
